# Professional Cleaning Salesperson Assignment
## Data Collection
Importing libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
from pandas.io.json import json_normalize # tranforming json file into a pandas dataframe library

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
print('Folium installed')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Folium installed


In [24]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim  # module to convert an address into latitude and longitude values
print('Geopy installed')
print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Geopy installed
Libraries imported.


Setting the Foursquare API credentials

In [10]:
# The code was removed by Watson Studio for sharing.

<b>Setting the parameters to search for venues</b>

In this part is deffined the area of search for venues indicated by latitude and longitute coordinates.
The grid size sets the individual area for each request to Foursquare API

In [31]:
grid_size_h = 15
grid_size_w = 22
north_limit = 45.448042
south_limit = 45.310282
west_limit = -75.845952
east_limit = -75.564285

grid_height = (north_limit - south_limit)/grid_size_h
grid_width = (west_limit - east_limit)/grid_size_w
search_query = 'Restaurant'
intent = 'browse'
search_params = {
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'intent': intent,
    'query': search_query,
    'limit': 100,
    'v': VERSION
}

geolocator = Nominatim(user_agent="Foursquare")
search_count = 0

<b>Searching for venues at Foursquare</b>

In this cell is the for loop to request venues in individual areas. Each areas has approximatelly 1km2.
The dataset is build in this cell as well. So for each request the result is appended on the end of the dataframe.

In [14]:
dataframe = []
for lat in range(grid_size_h):
    for lng in range(grid_size_w):
        ne_lat = north_limit - (lat * grid_height)
        ne_long = west_limit - (lng * grid_width)
        se_lat = ne_lat - grid_height
        se_lng = ne_long + grid_width

        search_params.update({'ne': '{},{}'.format(ne_lat, ne_long),
                              'sw': '{},{}'.format(se_lat, se_lng)})
        
        results = requests.get('https://api.foursquare.com/v2/venues/search',params=search_params)
        if 'venues' in results.json()['response']:
            venues = results.json()['response']['venues']

            for venue in venues:
                if len(dataframe) == 0:
                    dataframe = json_normalize(venue)
                dataframe2 = json_normalize(venue)
                dataframe = pd.concat([dataframe,dataframe2])

        search_count += 1

        if search_count % 30 == 0:
            print('Searched: {}'.format(search_count))

Searched: 30
Searched: 60
Searched: 90
Searched: 120
Searched: 150
Searched: 180
Searched: 210
Searched: 240
Searched: 270
Searched: 300
Searched: 330


In [15]:
len(dataframe) #Chequing if the size of the dataframe matches with the number of venues

180

<b>Formating the columns names</b>

In [16]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered = dataframe_filtered[['id','name','lat','lng','address','city','state','categories']]

<b>Removing duplicate inputs and inputs from Quebec</b>

In [17]:
dataframe_filtered.drop_duplicates(subset='id', keep='first', inplace=True)
dataframe_filtered = dataframe_filtered[dataframe_filtered.state != 'QC']

In [18]:
dataframe_filtered.reset_index(inplace=True) #Reseting index

<b>Removing NaN</b>

Some data is missing on address column, so, we can take the information of geographic coordinates to get the address of those venues.
The values with 'NaN' is substituted by the address found on the coordinates.

In [32]:
for i in dataframe_filtered.index[pd.isnull(dataframe_filtered['address']) == True].tolist():
    location = (geolocator.reverse(str(dataframe_filtered.loc[i]['lat']) +','+ str(dataframe_filtered.loc[i]['lng'])))
    local = location[0].split(',')
    address = str(local[0]) + ',' + str(local[1])
    city  = local[-4]
    if str(local[-3]) == ' Ontario':
        province = 'ON'
    elif str(local[-3]) == ' Québec':
        province = 'QC'
    else:
        province = local[-3]
    dataframe_filtered.at[i, 'address'] = str(address)
    dataframe_filtered.at[i, 'city'] = str(city)
    dataframe_filtered.at[i, 'state'] = str(province)
    print(i, dataframe_filtered.loc[i]['lat'], dataframe_filtered.loc[i]['lng'], address, city, province)

3 45.4393842495 -75.6273699762 La Cité collégiale, 801  Ottawa ON
5 45.42997 -75.69294 111, Murray Street  Ottawa ON
19 45.422136 -75.69912 157, Sparks Street  Ottawa ON
28 45.427654 -75.690132 372, Dalhousie Street  Ottawa ON
35 45.424597646 -75.6347759162 1054, Ogilvie Road  Ottawa ON
36 45.424679 -75.636918 Sleep Country, 1133  Ottawa ON
51 45.4193349434 -75.6996723544 129, Bank Street  Ottawa ON
72 45.418283 -75.612438 1552, Cyrville Road  Ottawa ON
74 45.409776 -75.77777 DoubleTree by Hilton Hotel Gatineau-Ottawa, 1170  Outaouais QC
96 45.4072064 -75.6862189 35, Linden Terrace  Ottawa ON
103 45.394624 -75.755414 322, Churchill Avenue North  Ottawa ON
112 45.3870584993 -75.7327157153 Carling TS, 1275  Ottawa ON
124 45.378407 -75.645932 1729, Walkley Road  Ottawa ON
125 45.3663488834 -75.7831763038 Lincoln Heights, Britannia Bay  Ottawa ON
139 45.355285 -75.805633 Jambo Restaurant, 69  Ottawa ON
143 45.3500088444 -75.7575040007 Algonquin College, 1385  Ottawa ON
147 45.353888 -75.65

<b>Removing any input that is not from Ontario

In [33]:
dataframe_filtered = dataframe_filtered[dataframe_filtered.state == 'ON']

In [34]:
dataframe_filtered.shape

(160, 9)

In [35]:
dataframe_filtered

,index,id,name,lat,lng,address,city,state,categories
1,0,4c0a619d7e3fc928ae70f382,McDonald's,45.441750,-75.643841,594 Montreal Rd.,Ottawa,ON,Fast Food Restaurant
2,0,4ba405edf964a520d07838e3,Pilo's Greek Restaurant,45.446167,-75.630297,876 Montreal Rd,Ottawa Division,ON,Greek Restaurant
3,0,52d71865498e489acd17525d,Le restaurant-école Les Jardins de la Cité,45.439384,-75.627370,"La Cité collégiale, 801",Ottawa,ON,Restaurant
4,0,4b5f23d4f964a5208ea829e3,Navarra,45.429785,-75.693345,93 Murray St.,Ottawa,ON,Spanish Restaurant
5,0,58c098aa01f43341813efdd0,Sur-Lie,45.429970,-75.692940,"111, Murray Street",Ottawa,ON,French Restaurant
6,0,4b5347adf964a520949527e3,Saffron Restaurant,45.430578,-75.681453,426 Rideau St.,Ottawa,ON,Middle Eastern Restaurant
7,0,4b9c3786f964a520e95536e3,Sitar Indian Restaurant,45.430496,-75.681453,417 Rideau St.,Ottawa,ON,Indian Restaurant
8,0,4b6f374bf964a520bde42ce3,Habesha,45.432539,-75.676591,574 Rideau St.,Ottawa,ON,Ethiopian Restaurant
9,0,4b61f2c4f964a520ea2c2ae3,Maskali Restaurant,45.429805,-75.665091,27 Selkirk St,Ottawa,ON,African Restaurant
10,0,4ec835a58b81dcfdc45e4b48,Best Pizza and Restaurants,45.431778,-75.655870,295 McArthur Ave,Vanier,ON,Pizza Place


In [39]:
address = '42 York St, Ottawa, ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) # generate map centred around the CEF

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=3,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map